In [1]:
%config Completer.use_jedi = False
import warnings
warnings.filterwarnings(action='ignore')
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


<img src="./images/MNIST.png" width="800"/>

MNIST 손글씨 실습을 위해서 케라스에서 제공하는 MNIST 데이터셋을 사용한다.

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

학습 데이터에는 총 60,000개의 샘플이 있고 테스트 데이터에는 총 10,000개의 샘플이 있다.
MNIST 데이터는 이미지 하나가 28개의 행과 28개의 열을 갖는 픽셀 데이터이다. 각 픽셀은 흑백 사진과 같이 0부터 255까지의 그레이스케일을 가지고 있다.

In [3]:
print('x_train.shape: {}, y_train.shape: {}'.format(x_train.shape, y_train.shape)) # 학습 데이터
print('x_test.shape: {}, y_test.shape: {}'.format(x_test.shape, y_test.shape)) # 테스트 데이터

x_train.shape: (60000, 28, 28), y_train.shape: (60000,)
x_test.shape: (10000, 28, 28), y_test.shape: (10000,)


학습 데이터를 학습 데이터(5만개)와 검증 데이터(1만개)로 분리한다.
학습 중간마다 검증 데이터로 모델의 성능을 측정하면 모델 학습이 제대로 진행되는지 검증 정확도를 알 수 있고 학습 정확도는 올라가는데 검증 정확도가 더이상 올라가지 않거나 떨어질 경우 조기 종료를 구현할 수 있다.

In [4]:
x_val = x_train[50000:] # 28행 28열로 구성된 검증 데이터
x_train = x_train[:50000] # 28행 28열로 구성된 학습 데이터
y_val = y_train[50000:] # 검증 데이터 실제값
y_train = y_train[:50000] # 학습 데이터 실제값
print('x_train.shape: {}, y_train.shape: {}'.format(x_train.shape, y_train.shape)) # 학습 데이터
print('x_val.shape: {}, y_val.shape: {}'.format(x_val.shape, y_val.shape)) # 검증 데이터
print('x_test.shape: {}, y_test.shape: {}'.format(x_test.shape, y_test.shape)) # 테스트 데이터

x_train.shape: (50000, 28, 28), y_train.shape: (50000,)
x_val.shape: (10000, 28, 28), y_val.shape: (10000,)
x_test.shape: (10000, 28, 28), y_test.shape: (10000,)


학습 데이터를 출력해보면 데이터가 0부터 255까지의 숫자(그레이스케일)로 구성된 것을 확인할 수 있다.

In [5]:
print(y_train[:10])
for i in x_train[1]:
    for j in i:
        print('{:3d} '.format(j), end='')
    print()

[5 0 4 1 9 2 1 3 1 4]
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  51 159 253 159  50   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0   0  48 238 252 252 252 237   0   0   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0   0   0  54 227 253 252 239 233 252  57   6   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0  10  60 224 252 253 252 202  84 252 253 122   0   0   0   0   0   0 
  0   0   0   0   0   0   0   0   0   0   0 163 252 252 252 253 252 252  9

MNIST 데이터를 불러왔으니 다층 퍼셉트론의 입력값으로 들어갈 수 있도록 numpy의 reshape() 함수를 사용해서 2차원 배열 형태의 데이터를 1차원 배열 형태로 변경한다.

In [6]:
# 28 * 28 픽셀의 단색 이미지이므로 데이터 형태를 784개의 1차원 배열 형태로 변환한다.
print('x_train.shape: {}'.format(x_train.shape))
# x_train = x_train.reshape(50000, 784)
x_train = np.reshape(x_train, [50000, 784]) # 28행 28열로 구성된 학습 데이터를 784개의 1차원 배열 형태로 변환한다.
print('x_train.shape: {}'.format(x_train.shape))
x_val = np.reshape(x_val, [10000, 784]) # 28행 28열로 구성된 검증 데이터를 784개의 1차원 배열 형태로 변환한다.
print('x_val.shape: {}'.format(x_val.shape))
x_test = np.reshape(x_test, [10000, 784]) # 28행 28열로 구성된 테스트 데이터를 784개의 1차원 배열 형태로 변환한다.
print('x_test.shape: {}'.format(x_test.shape))

x_train.shape: (50000, 28, 28)
x_train.shape: (50000, 784)
x_val.shape: (10000, 784)
x_test.shape: (10000, 784)


1차원으로 변경된 데이터를 그대로 다층 퍼셉트론에 입력해도 되지만 조금 더 효율적인 학습을 위해 데이터를 정규화 시킨다. 정규화는 모델의 학습 시간을 단축시키고 더 나은 성능을 보이게 하는 효과가 있다.
MNIST 데이터의 모든 값들은 0부터 255까지의 범위 안에 있으므로 255로 나눠 모든 값들을 0부터 1사이의 값으로 정규화 한다.

In [7]:
# print(type(x_train[0][0]), x_train[0][0])
# 255로 나눠서 0과 1사이의 실수로 만들어야 하므로 astype() 함수를 이용해서 정수를 실수로 변환한다.
x_train = x_train.astype('float32')
# print(type(x_train[0][0]), x_train[0][0])
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale
set(y_train)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

MNIST 손글씨 데이터 분류 모델은 0부터 9까지의 숫자로 분류하는 다중 모델이므로 손실 함수로 크로스 엔트로피를 사용한다.
크로스 엔트로피를 계산하기 위해 실제값(y_train, y_val, y_test)을 one-hot encoding으로 변환한다.
one-hot encoding은 데이터를 수만은 0과 1개의 1로 데이터를 구별하는 인코딩 방식으로 0으로 이루어진 벡터 집합에 단 1개의 1의 값으로 해당 데이터를 구별하는 것을 말한다.

In [8]:
# to_categorical() 함수로 데이터에 one-hot encoding을 적용한다.
num_classes = 10
print(y_train[:5])
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
for i in y_train[:5]:
    print(i)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

[5 0 4 1 9]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


<img src="./images/MNIST2.png" width="1800"/>

입력 데이터는 784개의 숫자가 들어있는 배열이다.
784개의 입력을 받는 256개의 노드가 1번째 히든 레이어에 있고 1번째 히든 레이어어의 출력값을 입력으로 받는 2번째 히든 레이어에는 128개의 노드가 있다.
2번째 히든 레이어에는 과대 적합을 방지하기 위해 10% 드롭아웃을 적용한다.
3번째 히든 레이어에서는 총 10개의 노드가 존재하며 이 10개의 노드값은 소프트맥스를 통과해서 0부터 9까지에 해당되는 각 숫자의 확률을 의미한다.
소프트맥스는 분류해야하는 정답지(레이블, 클래스, 타겟)의 총 개수를 k라고 할 때 k차원의 벡터를 입력받나 각 정답에 대한 확률을 추정한다.
소프트맥스의 출력값과 실제값의 차이(오차)를 계산하기 위해 크로스 엔트로피를 손실 함수로 사용하고 손실 함수를 최소화하기 위해 Adam 옵티마이저(최적화 함수)를 사용해 역전파를 통해 모든 가중치 및 편향값을 최적화한다.
최적화 함수 참고
https://onevision.tistory.com/entry/Optimizer-%EC%9D%98-%EC%A2%85%EB%A5%98%EC%99%80-%ED%8A%B9%EC%84%B1-Momentum-RMSProp-Adam

In [9]:
# 1번째 차원이 None인 이유는 데이터 개수의 제약없이 입력받기 위해서이고 2번째 차원이 784인 것은 MNIST의 이미지 크기가
# 28 * 28 픽셀 = 784 픽셀이기 때문이다.
x = tf.placeholder(dtype=tf.float32, shape=[None, 784]) # 학습 데이터를 기억할 placeholder를 선언한다.
y = tf.placeholder(dtype=tf.float32, shape=[None, 10]) # 실제값을 기억할 placeholder를 선언한다.
keep_prob = tf.placeholder(dtype=tf.float32) # 드롭아웃 적용 변수

다층 퍼셉트론을 구현한다.

In [10]:
def mlp(x):
    # 히든 레이어1
    w1 = tf.Variable(tf.random_uniform([784, 256]))
    b1 = tf.Variable(tf.zeros([256]))
    h1 = tf.nn.relu(tf.matmul(x, w1) + b1)
    # 히든 레이어2
    w2 = tf.Variable(tf.random_uniform([256, 128]))
    b2 = tf.Variable(tf.zeros([128]))
    h2 = tf.nn.relu(tf.matmul(h1, w2) + b2)
    # 드롭아웃 적용, keep_prob만 사용했었는데 버전이 올라가면서 rate=1-keep_prob과 같은 형태로 사용해야 한다.
    h2_drop = tf.nn.dropout(h2, rate=1-keep_prob)
    # 히든 레이어3
    w3 = tf.Variable(tf.random_uniform([128, 10]))
    b3 = tf.Variable(tf.zeros([10]))
    logits = tf.nn.relu(tf.matmul(h2_drop, w3) + b3)
    return logits

다층 퍼셉트론의 출력값을 logits(logistic + probit)로 정의한다. probit는 확률을 재는 단위를 의미한다.

In [11]:
logits = mlp(x)
# logits와 실제값의 크로스 엔트로피를 손실 함수로 사용한다.
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=y)
)
# Adam 옵티마이저를 사용해 모델을 최적화한다.
# 모델의 최적화 과정은 모델의 예측값과 실제값의 차이를 줄여나가는 과정을 의미한다.
train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

조기 종료는 과대적합을 피하고 충분한 학습을 하기 위해서 학습 중간마다 검증 데이터에 대한 정확도를 측정하면 학습 데이터에 대한 정확도는 계속 증가하는 반면 검증 데이터에 대한 정확도가 점차 떨어질 경우 학습을 중지하는 것을 말한다.
매 epoch 마다 검증 데이터로 검증 정확도를 측정해서 검증 정확도가 5번 연속으로 검증 정확도의 최대값보다 높지 않을 경우 조기 종료를 수행한다.

In [12]:
init = tf.global_variables_initializer() # 텐서플로우 변수 초기화
saver = tf.train.Saver() # 텐서플로우에서 학습 모델의 저장 및 로드에 사용할 변수를 선언한다.
epoch_cnt = 300 # 조기 종료가 일어나지 않을 경우 최대 300번까지 반복해서 학습하도록 설정한다.
batch_size = 1000 # 1번에 읽어서 처리할 데이터의 개수를 설정한다.
iteration = len(x_train) // batch_size # batch_size에 따른 학습 횟수를 설정한다.
earlystop_threshold = 5 # 검증 정확도가 검증 정확도의 최대값보다 5번 연속으로 높지 않을 경우 조기 종료하도록 설정한다.
earlystop_cnt = 0 # 검증 정확도가 검증 정확도의 최대값보다 연속으로 높지 않은 검증 횟수를 세는 변수를 선언한다.

학습을 진행한다.

In [13]:
a = tf.constant([3, 10, 5]) # 1차원 배열
sess = tf.Session()
print(sess.run(a))
# argmax() 함수는 배열에서 가장 큰 값을 찾아서 인덱스를 리턴한다.
# a 배열에서 10이 가장 크기 때문에 결과는 10의 인덱스인 1이 출력된다.
print(sess.run(tf.argmax(a)))

Metal device set to: Apple M2
[ 3 10  5]
1


2023-03-05 14:33:36.611488: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-05 14:33:36.611699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-05 14:33:36.615718: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-03-05 14:33:36.622491: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-03-05 14:33:36.628687: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-05 14:33:36.657624: I tensorflow/core/grappler/optimizers/custom_graph_

In [14]:
b = tf.constant([[3, 10, 5], [4, 5, 6], [0, 8, 7]]) # 2차원 배열
print(sess.run(b))
# argmax() 함수를 2차원 배열에서 사용할 경우 2번째 인수를 지정해야 한다. 생략시 기본값은 0
# 2번째 인수를 생략하거나 0을 사용하면 2차원 배열의 각 열에서 최대값의 인덱스를 리턴하고 2번째 인수에 1을 쓰면 2차원 배열의
# 각 행에서 최대값의 인덱스를 리턴한다.
# b 배열에서 열 단위로 최대값을 계산하면 0번째 열의 최대값 4의 인덱스 1, 1번째 열의 최대값 10의 인덱스 0, 2번째 열의 최대값
# 7의 인덱스 2가 [1 0 2]와 같이 출력된다.
print(sess.run(tf.argmax(b)))
print(sess.run(tf.argmax(b, 0)))
# b 배열의 행 단위로 최대값을 계산하면 0번째 행의 최대값 10의 인덱스 1, 1번째 행의 최대값 6의 인덱스 2, 2번째 행의 최대값
# 8의 인덱스 1이 [1 2 1]과 같이 출력된다.
print(sess.run(tf.argmax(b, 1)))

[[ 3 10  5]
 [ 4  5  6]
 [ 0  8  7]]
[1 0 2]
[1 0 2]
[1 2 1]


2023-03-05 14:33:36.667842: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-05 14:33:36.672071: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-05 14:33:36.677807: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-05 14:33:36.681670: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [15]:
with tf.Session() as sess:
    sess.run(init)
    prev_train_acc = 0.0 # 이전 학습 정확도를 기억할 변수를 선언한다.
    max_val_acc = 0.0 # 검증 정확도의 최대값을 기억할 변수를 선언한다.
    # 지정한 최대 epoch 만큼 학습한다. => 검증 정확도가 검증 정확도의 최대값보다 5번 연속으로 높지 않을 경우 조기 종료한다.
    for epoch in range(epoch_cnt):
        avg_loss = 0.0 # 손실값
        start = 0 # 학습 시작 위치
        end = batch_size # 학습 끝 위치
        # 학습 데이터를 batch_size 개 만큼씩 나눠서 학습을 진행한다.
        for i in range(iteration):
            _, loss_op = sess.run([train, loss], feed_dict={x: x_train[start:end], y: y_train[start:end], keep_prob: 0.9})
            # 학습할 데이터의 범위를 batch_size 만큼 이동시킨다.
            start += batch_size
            end += batch_size
            # 크로스 엔트로피 손실 함수의 학습 손실값을 계산한다.
            avg_loss += loss_op / iteration
        # ===== for i

        # 모델 검증
        predict = tf.nn.softmax(logits=logits) # 소프트맥스 적용
        correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(y, 1))

        # 정확도 계산
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
        # 학습 정확도 계산
        # Session.run()과 Tansor.eval()의 차이
        # t가 Trnsor 오브젝트라면 t.eval()은 sess.run(t)의 속기 표현이다. sess가 현재 디폴트 세션인 곳에서만 가능하다.
        cur_train_acc = accuracy.eval({x: x_train, y: y_train, keep_prob: 1.0})
        # 검증 정확도 계산
        cur_val_acc = accuracy.eval({x: x_val, y: y_val, keep_prob: 1.0})
        # 검증 데이터에 대한 손실값 계산
        cur_val_loss = loss.eval({x: x_val, y: y_val, keep_prob: 1.0})
        # 학습 정확도와 검증 정확도를 출력한다.
        print('epoch: {:3d}, 학습 정확도: {:7.5f}, 검증 정확도: {:7.5f}'.format(epoch, cur_train_acc, cur_val_acc))

        # 현재 검증 정확도와 검증 정확도의 최대값을 비교한다.
        if cur_val_acc < max_val_acc:
            # 현재 검증 정확도가 검증 정확도의 최대값 미만이면
            # 현재 학습 정확도와 이전 학습 정확도, 현재 학습 정확도와 0.99를 비교한다.
            if cur_train_acc > prev_train_acc or cur_train_acc > 0.99:
                # 현재 학습 정확도가 이전 학습 정확도 보다 크거나 현재 학습 정확도가 0.99 보다 크면
                if earlystop_cnt == earlystop_threshold:
                    print('조기 종료 시점: {}'.format(epoch))
                    break
                else:
                    # 5번 연속으로 현재 학습 정확도가 이전 학습 정확도 보다 크지 않다면 이전 학습 정확도가 현재 학습 정확도
                    # 보다 크므로 현재 검증 정확도가 검증 정확도의 최대값보다 연속으로 높지 않은 횟수를 카운트 하는 변수
                    # earlystop_cnt를 1증가 시킨다.
                    earlystop_cnt += 1
                    print('과대적합 경고 횟수: {}'.format(earlystop_cnt))
                # ===== if
            else:
                # 현재 학습 정확도가 이전 학습 정확도 이하 이거나 현재 학습 정확도가 0.99 보다 이하이면
                # 이전 학습 정확도가 현재 학습 정확도 이상이므로 현재 검증 정확도가 검증 정확도의 최대값보다 연속으로
                # 높지 않은 횟수를 카운트 하는 변수를 0으로 초기화시킨다.
                earlystop_cnt = 0
            # ===== if
        else:
            # 현재 검증 정확도가 검증 정확도의 최대값 이상이면
            # 현재 검증 정확도가 검증 정확도의 최대값 보다 연속을 높지 않은 횟수를 기억하는 변수 earlystop_cnt를
            # 0으로 초기화시킨다.
            earlystop_cnt = 0
            # 검증 정확도의 최대값을 현재 검증 정확도로 교체한다.
            max_val_acc = cur_val_acc
            # 검증 정확도가 가장 높은 현재 모델을 저장한다.
            save_path = saver.save(sess, './model/model.ckpt')
        # ===== if
        # 다음 학습을 위해 현재 학습 정확도를 이전 학습 정확도를 기억하는 변수에 넣어준다.
        prev_train_acc = cur_train_acc
    # ===== for
# ===== with

2023-03-05 14:33:36.687682: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-05 14:33:36.687690: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-05 14:33:36.690454: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-05 14:33:36.780447: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-05 14:33:38.342939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-05 14:33:38.78

epoch:   0, 학습 정확도: 0.13848, 검증 정확도: 0.14570


2023-03-05 14:33:39.281373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


epoch:   1, 학습 정확도: 0.58388, 검증 정확도: 0.60380


2023-03-05 14:33:39.748043: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


...

epoch:  84, 학습 정확도: 0.97278, 검증 정확도: 0.94720
과대적합 경고 횟수: 4


2023-03-05 14:34:27.636809: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


epoch:  85, 학습 정확도: 0.97286, 검증 정확도: 0.94720
과대적합 경고 횟수: 5


2023-03-05 14:34:28.100236: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


epoch:  86, 학습 정확도: 0.97322, 검증 정확도: 0.94720
조기 종료 시점: 86


검증 결과가 가장 높았던 모델을 불러온다.

In [16]:
with tf.Session() as sess:
    saver.restore(sess, './model/model.ckpt')
    correct_prediction = tf.equal(tf.argmax(predict, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))
    print('정확도: {}'.format(accuracy.eval({x: x_train, y: y_train, keep_prob: 1.0})))

INFO:tensorflow:Restoring parameters from ./model/model.ckpt
정확도: 0.971019983291626


2023-03-05 14:34:29.184139: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-05 14:34:29.184157: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-03-05 14:34:29.195697: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-05 14:34:29.221070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


<img src="./images/MNIST3.png" width="1800"/>